In [11]:
import requests
import json

In [12]:
with open("../secrets/lastfm.json") as f:
    client_json = json.load(f)

In [13]:
base_url = "http://ws.audioscrobbler.com/2.0/"

# test to see if we get response
headers = {
    "user-agent": "python-tester"
}

payload = {
    "api_key": client_json["client_id"],
    "method": "chart.gettopartists",
    "format": "json"
}

r = requests.get(base_url, headers=headers, params=payload)
r.status_code

200

In [5]:
# test to get auth token
auth_payload = {
    "api_key": client_json["client_id"],
    "method": "auth.gettoken",
    "format": "json"
}

auth_r = requests.get(base_url, headers=headers, params=auth_payload)
auth_r.status_code

200

In [6]:
auth_token = auth_r.json()["token"]
print(auth_token)

_ALY6pZySTcPoZ-a1MKgqV2J4ppQCQyr


In [ ]:
# authenticate user?
import webbrowser

auth_request_url = "http://www.last.fm/api/auth"
request_auth_payload = {
    "api_key": client_json["client_id"],
    "token": auth_token
}

request_auth_r = requests.get(auth_request_url, headers=headers, params=request_auth_payload)
request_auth_r.status_code

webbrowser.open(request_auth_r.url)

True

In [ ]:
# signing calls, needed for use of services
import hashlib
auth_payload = {
    "api_key": client_json["client_id"],
    "method": "auth.gettoken",
    "format": "json"
}
request_params_signing = "".join([str(k)+str(v) for k, v in auth_payload.items()][:-1]) + client_json["client_secret"]
print(request_params_signing)
print(hashlib.md5(request_params_signing.encode()))

# signing needs
    # - api key
    # - api secret
    # - method
# join "api_key"+api_key-value, "method"+method-string and api_secret

api_key96e8b6dd8a4702e9e058ed121094987fmethodauth.gettoken7f65cc958a0fe5800a535207c43fbcc9
<md5 _hashlib.HASH object @ 0x105da9bb0>


In [ ]:
[str(k)+str(v) for k, v in auth_payload.items()][:-1]

['api_key96e8b6dd8a4702e9e058ed121094987f', 'methodauth.gettoken']

lol getRecentTracks trenger ikke auth.

In [15]:
import pytz
from datetime import datetime, timedelta
import webbrowser
import requests
today = pytz.utc.localize(datetime.now().replace(hour=0, minute=0, second=0, microsecond=0))
yesterday = today - timedelta(days=5)
today_ms = int(today.timestamp())
yesterday_ms = int(yesterday.timestamp())

base_url = "http://ws.audioscrobbler.com/2.0/"

headers = {
    "user-agent": "GP-personal-app"
}

recent_tracks_params = {
    "limit": 20,
    "user": "GammelPerson",
    "from": yesterday_ms,
    "to": today_ms,
    "extended": 0,
    "api_key": client_json["client_id"],
    "format": "json",
    "method": "user.getrecenttracks",
}

r = requests.get(base_url, headers=headers, params=recent_tracks_params)
webbrowser.open(r.url)
r.status_code

200

In [ ]:
r.url

'http://ws.audioscrobbler.com/2.0/?limit=20&user=GammelPerson&from=1764115200&to=1764547200&extended=1&api_key=96e8b6dd8a4702e9e058ed121094987f&format=json&method=user.getrecenttracks'

In [ ]:
date_format = "%Y-%m-%d"
end = "2025-12-01"
int(
        pytz.utc.localize(datetime.strptime(end, date_format)).timestamp()
    )


1764547200

In [ ]:

webbrowser.open(r.url)


True

Response er en .json-fil. Den har to nøkler på høyeste nivå, `recent_tracks` og `@attr`. Gitt at du gjør et søk over en lenger periode, for eksempel her med flere dagers mellomrom, er det mulig å få antall sider med resultater i `@attr`. I `recent_tracks.tracks` finner man da all sanginfoen.

Det går vel an å bruke dette for å hente hele lyttehistorien min fra 01.01-30.11. Bare at jeg kanskje burde indeksere sidene og dumpe resultatene til json. Vet ikke om dette overskrider rate_limit satt av last.fm.

In [ ]:
r.url

'http://ws.audioscrobbler.com/2.0/?limit=20&user=GammelPerson&from=1764288000000&to=1764374400000&extended=1&api_key=96e8b6dd8a4702e9e058ed121094987f&format=json&method=user.getrecenttracks'

In [ ]:
q = r.json()

In [ ]:
q["recenttracks"]["track"][0]["artist"]["mbid"]

''

In [ ]:
q

{'recenttracks': {'track': [{'artist': {'mbid': '', '#text': 'Slimesito'},
    'streamable': '0',
    'image': [{'size': 'small',
      '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/a9198665ef7ca70d920e25c509d322ba.jpg'},
     {'size': 'medium',
      '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/a9198665ef7ca70d920e25c509d322ba.jpg'},
     {'size': 'large',
      '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/a9198665ef7ca70d920e25c509d322ba.jpg'},
     {'size': 'extralarge',
      '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/a9198665ef7ca70d920e25c509d322ba.jpg'}],
    'mbid': '',
    'album': {'mbid': '', '#text': 'Pray 4 Me'},
    'name': 'Pray 4 Me',
    'url': 'https://www.last.fm/music/Slimesito/_/Pray+4+Me',
    'date': {'uts': '1764531747', '#text': '30 Nov 2025, 19:42'}},
   {'artist': {'mbid': 'ec29539f-4e72-46af-9fb0-b5697a3cf41b',
     '#text': 'Evita Manji'},
    'streamable': '0',
    'image': [{'size': 'small',
      '#text': 'https://lastfm

In [ ]:
q["recenttracks"]["track"]

[{'artist': {'mbid': '', '#text': 'Slimesito'},
  'streamable': '0',
  'image': [{'size': 'small',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/a9198665ef7ca70d920e25c509d322ba.jpg'},
   {'size': 'medium',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/a9198665ef7ca70d920e25c509d322ba.jpg'},
   {'size': 'large',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/a9198665ef7ca70d920e25c509d322ba.jpg'},
   {'size': 'extralarge',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/a9198665ef7ca70d920e25c509d322ba.jpg'}],
  'mbid': '',
  'album': {'mbid': '', '#text': 'Pray 4 Me'},
  'name': 'Pray 4 Me',
  'url': 'https://www.last.fm/music/Slimesito/_/Pray+4+Me',
  'date': {'uts': '1764531747', '#text': '30 Nov 2025, 19:42'}},
 {'artist': {'mbid': 'ec29539f-4e72-46af-9fb0-b5697a3cf41b',
   '#text': 'Evita Manji'},
  'streamable': '0',
  'image': [{'size': 'small',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/49d913353ae9b30e4df493b16613274a.jpg'

In [ ]:
pytz.utc.localize(datetime(year=2025, month=1, day=1, hour=0, minute=0, second=0))

datetime.datetime(2025, 1, 1, 0, 0, tzinfo=<UTC>)

In [10]:
# Sjekke hvor mange .json filer jeg må dumpe hvis vi henter 50 sanger per request.

import pytz
from datetime import datetime, timedelta
import webbrowser
start = pytz.utc.localize(datetime(year=2020, month=4, day=1, hour=0, minute=0, second=0))
end = pytz.utc.localize(datetime(year=2025, month=12, day=17, hour=0, minute=0, second=0))
start_ms = int(start.timestamp())
end_ms = int(end.timestamp())
# start_ms = int(start.timestamp() * 100)
# end_ms = int(end.timestamp() * 100)

base_url = "http://ws.audioscrobbler.com/2.0/"

recent_tracks_params = {
    "limit": 200,
    "user": "GammelPerson",
    "from": start_ms,
    "to": end_ms,
    "extended": 0,
    "api_key": client_json["client_id"],
    "format": "json",
    "method": "user.getrecenttracks",
    "page": 1,
}

r = requests.get(base_url, headers=headers, params=recent_tracks_params)
webbrowser.open(r.url)
r.status_code

print(r.json()["recenttracks"]["@attr"])

{'perPage': '200', 'totalPages': '515', 'page': '1', 'user': 'GammelPerson', 'total': '102823'}


In [18]:
webbrowser.open(r.url)

True

I perioden 01.01-30.11 blir det totalt 321 sider, eller 321 .json-filer totalt.

Kan lage en cron-jobb som tar og samler inn data fra foregående dag.

In [ ]:
qe = datetime.now()

In [ ]:
qe.strftime("%B").lower()

'december'

In [ ]:
number = 1
print("{:04d}".format(number))


0001


In [ ]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()
logger.info("Crap")

2025-12-01 17:54:58,430 - INFO - Crap


In [13]:
import pytz
from datetime import datetime, timedelta
import webbrowser
import requests
today = pytz.utc.localize(datetime.now().replace(hour=0, minute=0, second=0, microsecond=0))
yesterday = today - timedelta(days=5)
today_ms = int(today.timestamp())
yesterday_ms = int(yesterday.timestamp())

base_url = "http://ws.audioscrobbler.com/2.0/"

headers = {
    "user-agent": "GP-personal-app"
}

recent_tracks_params = {
    "autocorrect": 1,
    "api_key": client_json["client_id"],
    "format": "json",
    "method": "album.getTags",
    "user": "GammelPerson",
    "artist": "Playboi Carti",
    "album": "Music"
}

r = requests.get(base_url, headers=headers, params=recent_tracks_params)
webbrowser.open(r.url)
r.status_code

200

In [ ]:
import pathlib
import json
import logging
config_file = pathlib.Path("logging_config/config.json")
with open(config_file) as f_in:
    config = json.load(f_in)
logging.config.dictConfig(config)

In [21]:
# Sjekke hvor mange .json filer jeg må dumpe hvis vi henter 50 sanger per request.

import pytz
from datetime import datetime, timedelta
start = pytz.utc.localize(datetime(year=2025, month=1, day=29, hour=0, minute=0, second=0))
end = pytz.utc.localize(datetime(year=2025, month=3, day=10, hour=0, minute=0, second=0))
start_ms = int(start.timestamp())
end_ms = int(end.timestamp())

base_url = "http://ws.audioscrobbler.com/2.0/"

recent_tracks_params = {
    "limit": 200,
    "user": "GammelPerson",
    "from": start_ms,
    "to": end_ms,
    "extended": 0,
    "api_key": client_json["client_id"],
    "format": "json",
    "method": "user.getrecenttracks",
    "page": 1,
}

r = requests.get(base_url, headers=headers, params=recent_tracks_params)
r.status_code

print(r.json()["recenttracks"]["@attr"])

{'perPage': '200', 'totalPages': '1', 'page': '1', 'user': 'GammelPerson', 'total': '160'}


In [22]:
r.json()["recenttracks"]

{'track': [{'artist': {'mbid': '757ed661-fbad-4e45-b1cd-a6b09f06f54a',
    '#text': 'Yung Lean'},
   'streamable': '0',
   'image': [{'size': 'small',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/c57c5b1e7326ec07d9b74122ec89953a.jpg'},
    {'size': 'medium',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/c57c5b1e7326ec07d9b74122ec89953a.jpg'},
    {'size': 'large',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/c57c5b1e7326ec07d9b74122ec89953a.jpg'},
    {'size': 'extralarge',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/c57c5b1e7326ec07d9b74122ec89953a.jpg'}],
   'mbid': '13083b88-2c5a-48f5-b238-2864ed6baa77',
   'album': {'mbid': '', '#text': 'Forever Yung'},
   'name': 'Forever Yung',
   'url': 'https://www.last.fm/music/Yung+Lean/_/Forever+Yung',
   'date': {'uts': '1741548248', '#text': '09 Mar 2025, 19:24'}},
  {'artist': {'mbid': '86f48c33-57de-4d86-87b3-7397d5e4baf4',
    '#text': 'Maria Somerville'},
   'streamable': '0',
   'im